# System setup

In [1]:
%matplotlib inline

import pymc3 as pm
import numpy as np
import pandas as pd
import scipy.stats as stats
import sys
import time

import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Helper functions

In [2]:
def progress_bar(value, endvalue, bar_length=20):
        percent = float(value) / endvalue
        arrow = '-' * int(round(percent * bar_length)-1) + '>'
        spaces = ' ' * (bar_length - len(arrow))
        sys.stdout.write("\rProgress: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
        sys.stdout.flush()

# Bayesian prospective power analysis

In [7]:
def power_analysis_bayes (n, target_power, increment, reps):
    estimated_power = 0.0
    result = []

    while estimated_power < target_power:
        print("N: " + str(n))

        # Select a value for the “true” bias in the coin, from the data-generating distribution and
        # subsequently simulate flipping a coin with that bias n times, i. e. simulate outcome variable.
        simulated_data = [np.random.binomial(1, bias, n) for bias in [np.random.beta(a=1300, b=700) for x in range(0, reps)]]

        for i in range(reps): # repeat with a feasible large number to get a good approximation of the power
            progress_bar(i, reps)

            # Using an audience-agreeable prior for purposes of data analysis, 
            # determine the posterior beliefs regarding θ if z heads 
            # in N flips were observed.
            with pm.Model() as model:        
                # Assume the audience-agreeable prior is uniform.
                p = pm.Uniform("p", 0, 1)

                # Likelihood of observations, i. e. sampling distribution of the response data.
                Y_obs = pm.Binomial("Y_obs", n=1, p=p, observed=simulated_data[i])

                # Model fitting, i. e. determine posterior distribution of model paramters.
                step = pm.Metropolis()
                trace = pm.sample(2000, step=step, progressbar=True)

                # Posterior analysis: Tally whether the 95% HDI excludes the null value of θ = .50, i. e.
                # smaller or equal than the lower boarder of the 95% HDI, i. e. 
                # exclude the 'null' value 0.5
                result.append(0.5 <=  pm.stats.hpd(trace)["p"][0])

        estimated_power = sum(result)/len(result)
        print()
        print("Estimated Power: " + str(estimated_power))
        print()
        if estimated_power < target_power: 
            n = n + increment


In [8]:
start_time = time.time()
power_analysis_bayes(n=60, increment=5, target_power=0.70, reps=5)
print("--- %s seconds ---" % (time.time() - start_time))

N: 60
Progress: [>                   ] 0%

100%|████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:00<00:00, 2528.81it/s]


Progress: [--->                ] 20%

100%|████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:00<00:00, 8480.95it/s]


Progress: [------->            ] 40%

...

Progress: [----------->        ] 60%

100%|████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:00<00:00, 7248.27it/s]


Progress: [--------------->    ] 80%

100%|████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:00<00:00, 6344.54it/s]



Estimated Power: 0.7

--- 39.99861693382263 seconds ---
